In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import re

In [5]:
ruta = ".\datos_mesagges.csv"
df = pd.read_csv(ruta, encoding = 'utf-8')
df.head()

,message_id,channel_id,user_id,text,created_at,user_name,channel_name,longitud_texto,hora_envio,dia_semana,sentimiento,es_pregunta,tiempo_hasta_respuesta,es_cuello_botella,falta_claridad
0,M0005,C001,U020,¡Feliz viernes equipo! 🎉,2025-06-18 08:17:49+00:00,Angelino Chacón (Data),general,24,8,2,0.0,False,1418.166667,0,0
1,M0017,C001,U009,¿Alguien sabe cuándo es la próxima demo? 🔥,2025-06-19 07:55:59+00:00,Teobaldo Carrasco (Frontend),general,42,7,3,0.0,True,385.116667,1,0
2,M0011,C001,U010,Recordatorio: reunión de retrospectiva mañana ✅,2025-06-19 14:21:06+00:00,América Manuel (UX/UI),general,47,14,3,0.0,False,600.800000,0,0
3,M0013,C001,U005,¡Hola equipo! ¿Cómo van con sus tareas? 💯,2025-06-20 00:21:54+00:00,Ofelia Torrents (DevOps),general,41,0,4,0.0,True,1336.633333,1,0
4,M0018,C001,U013,Recordatorio: reunión de retrospectiva mañana,2025-06-20 22:38:32+00:00,Anastasia Batlle (Frontend),general,45,22,4,0.0,False,475.316667,0,0


Creación del modelo: Identificar cuellos de botella en la comunicación

In [ ]:

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Elimina signos y pasa a minúsculas
    return text

text_col = 'text'
numeric_cols = ["tiempo_hasta_respuesta", "longitud_texto", "hora_envio", "dia_semana", "sentimiento", "es_pregunta"]


preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(preprocessor=clean_text, stop_words='spanish'), text_col),
        ('num', StandardScaler(), numeric_cols)
    ]
)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier())
])


X = df[[text_col] + numeric_cols]
y = df["es_cuello_botella"]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


pipeline = Pipeline([
    ('scaler', StandardScaler()),  
    ('model', RandomForestClassifier(class_weight='balanced'))
])

pipeline.fit(X_train, y_train)

ValueError: could not convert string to float: '¿Cómo manejan las validaciones de formularios?'

In [ ]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))